# CSC5610 Final - Water Potability Analysis

Authors: **Jacob Buysse**, **Andrew Cook**, **Josh Grant**

## Part 6 - Feature Engineering

For this we will be using the following libraries...

In [11]:
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Let us define a helper function for scoring models.

In [19]:
def score_model(test_y, pred_y):
    tn, fp, fn, tp = confusion_matrix(test_y, pred_y).ravel()
    print(f"True Positive {tp} (Correctly classified potable)")
    print(f"True Negative: {tn} (Correctly classified non-potable)")
    print(f"False Positive {fp} (Incorrectly classified non-potable as potable - Bad)")
    print(f"False Negative {fn} (Incorrectly classified potable as non-potable - Meh)")
    print(classification_report(test_y, pred_y, digits=5))

Let us load our potability data.

In [4]:
pdf = pd.read_feather("./potability.feather")
pdf.head()

,station_id,latitude,longitude,county_name,sample_code,potable_(Aminomethyl)phosphonic acid,"potable_1,1,1,2-Tetrachloroethane","potable_1,1,1-Trichloroethane","potable_1,1,2,2-Tetrachloroethane","potable_1,1,2-Trichloroethane",...,"param_p,p'-DDE","param_p,p'-DDT",param_p-Xylene,param_pH,"param_s,s,s-Tributyl Phosphorotrithioate (DEF)",param_sec-Butylbenzene,param_tert-Butylbenzene,"param_trans-1,2-Dichloroethene","param_trans-1,3-Dichloropropene",potable
0,1,38.5596,-121.4169,Sacramento,C0114B0005,True,True,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,1,38.5596,-121.4169,Sacramento,C0115B0005,True,True,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2,1,38.5596,-121.4169,Sacramento,C0116B0005,True,True,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,1,38.5596,-121.4169,Sacramento,C0117B0081,True,True,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,1,38.5596,-121.4169,Sacramento,C0118B0005,True,True,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


Let us load our county data.

In [5]:
cdf = pd.read_feather("./counties.feather")
cdf.head()

,CountyName,AdminRegion,FireMAR,LawMAR,Shape__Area,Shape__Length,emp,qp1,ap,est,...,n50_99,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4,Population
0,Alameda,Coastal,2,2,3.082165e+09,432059.180712,19808892,334583251,1276862745,1197073,...,32312.0,17613.0,3388.0,808.0,644.0,94.0,135.0,67.0,11.0,1628997
1,Alpine,Inland,4,4,3.145871e+09,274621.121060,25372729,358615051,1326420707,1473927,...,42557.0,23000.0,4396.0,1317.0,1079.0,289.0,280.0,166.0,59.0,1190
2,Amador,Inland,4,4,2.559998e+09,357482.565247,14140321,185837886,727112583,961824,...,21549.0,11581.0,2214.0,563.0,309.0,83.0,56.0,10.0,0.0,41412
3,Butte,Inland,3,3,7.338660e+09,526729.272631,7085896,94172636,348842612,477062,...,10146.0,5091.0,858.0,234.0,174.0,48.0,28.0,0.0,0.0,207303
4,Calaveras,Inland,4,4,4.352160e+09,371781.055548,8547832,105647215,421612466,591560,...,12189.0,6625.0,952.0,283.0,201.0,59.0,34.0,4.0,0.0,46563


Now let us merge the two.

In [33]:
df = pdf.merge(cdf, how="left", left_on="county_name", right_on="CountyName")
df.head()

,station_id,latitude,longitude,county_name,sample_code,potable_(Aminomethyl)phosphonic acid,"potable_1,1,1,2-Tetrachloroethane","potable_1,1,1-Trichloroethane","potable_1,1,2,2-Tetrachloroethane","potable_1,1,2-Trichloroethane",...,n50_99,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4,Population
0,1,38.5596,-121.4169,Sacramento,C0114B0005,True,True,True,True,True,...,19033.0,9932.0,1738.0,478.0,297.0,58.0,33.0,37.0,9.0,1584169
1,1,38.5596,-121.4169,Sacramento,C0115B0005,True,True,True,True,True,...,19033.0,9932.0,1738.0,478.0,297.0,58.0,33.0,37.0,9.0,1584169
2,1,38.5596,-121.4169,Sacramento,C0116B0005,True,True,True,True,True,...,19033.0,9932.0,1738.0,478.0,297.0,58.0,33.0,37.0,9.0,1584169
3,1,38.5596,-121.4169,Sacramento,C0117B0081,True,True,True,True,True,...,19033.0,9932.0,1738.0,478.0,297.0,58.0,33.0,37.0,9.0,1584169
4,1,38.5596,-121.4169,Sacramento,C0118B0005,True,True,True,True,True,...,19033.0,9932.0,1738.0,478.0,297.0,58.0,33.0,37.0,9.0,1584169


Let us start with the hypothesis that if we can indirectly determine the potability of each parameter without the direct measurement of that parameter then we could predict the overall potability without any direct label leakage.  Let us analyze the top offenders:

In [47]:
non_potable = df.potable.value_counts()[False]
print(f"{non_potable} non-potable samples.")
top_features = [
    "Dissolved Boron", "Dissolved Nitrate", "Dissolved Fluoride", "Bromodichloromethane", "Total Manganese",
    "Chloroform", "Dissolved Manganese", "Dissolved Selenium", "Dissolved Arsenic"
]
df["f"] = False
top_mask = df.f
for feature in top_features:
    top_count = df[f"potable_{feature}"].value_counts()[False]
    print(f"{top_count} due to {feature} ({top_count / non_potable:%})")
    top_mask = top_mask | ~df[f"potable_{feature}"]
non_potable_due_to_top = df[top_mask].potable.value_counts()[False]
print(f"{non_potable_due_to_top} non-potable samples based on just the top parameters ({non_potable_due_to_top / non_potable:%}).")

29573 non-potable samples.
8764 due to Dissolved Boron (29.635140%)
8292 due to Dissolved Nitrate (28.039090%)
4037 due to Dissolved Fluoride (13.650965%)
3467 due to Bromodichloromethane (11.723532%)
3005 due to Total Manganese (10.161296%)
2996 due to Chloroform (10.130863%)
2295 due to Dissolved Manganese (7.760457%)
1831 due to Dissolved Selenium (6.191458%)
2022 due to Dissolved Arsenic (6.837318%)
28123 non-potable samples based on just the top parameters (95.096879%).


Let us see if we can just predict the potability of the Dissolved Boron.

In [10]:
label_feature = "potable_Dissolved Boron"
df[label_feature] = df[label_feature].astype("category")
train_df, test_df = train_test_split(df, train_size=0.75, random_state=777, stratify=df[label_feature])
print(f"Train size: {train_df.shape}, Test size: {test_df.shape}")

Train size: (257433, 898), Test size: (85811, 898)


Let us scale our numeric features.

In [12]:
scaler = StandardScaler()
num_features = [
    "latitude", "longitude",
    "Shape__Area", "Shape__Length",
    "emp", "qp1", "ap", "est",
    "n<5", "n5_9", "n10_19", "n20_49", "n50_99", "n100_249", "n250_499", "n500_999",
    "n1000", "n1000_1", "n1000_2", "n1000_3", "n1000_4",
    "Population"
] + df.columns[df.columns.str.contains("param_")].values.tolist()
num_features.remove("param_Dissolved Boron")
scaler.fit(train_df[num_features])
train_num = scaler.transform(train_df[num_features])
test_num = scaler.transform(test_df[num_features])

Let us one-hot encode our categorical features.

In [16]:
hot_enc = OneHotEncoder()
df.county_name = df.county_name.astype("category")
cat_features = ["county_name", "AdminRegion", "FireMAR", "LawMAR"]
hot_enc.fit(train_df[cat_features])
train_hot = hot_enc.transform(train_df[cat_features])
test_hot = hot_enc.transform(test_df[cat_features])

And finally combine our X,y datasets for training and testing.

In [18]:
train_X = sp.hstack((train_hot, train_num))
train_y = train_df[label_feature].values
test_X = sp.hstack((test_hot, test_num))
test_y = test_df[label_feature].values
print(f"Train X {train_X.shape}, y {len(train_y)}")
print(f"Test X {test_X.shape}, y {len(test_y)}")

Train X (257433, 529), y 257433
Test X (85811, 529), y 85811


Let us test Logistic Regression

In [20]:
model = LogisticRegression(random_state=777, max_iter=10000)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
score_model(test_y, pred_y)

True Positive 83478 (Correctly classified potable)
True Negative: 1130 (Correctly classified non-potable)
False Positive 1061 (Incorrectly classified non-potable as potable - Bad)
False Negative 142 (Incorrectly classified potable as non-potable - Meh)
              precision    recall  f1-score   support

         0.0    0.88836   0.51575   0.65261      2191
         1.0    0.98745   0.99830   0.99285     83620

    accuracy                        0.98598     85811
   macro avg    0.93791   0.75702   0.82273     85811
weighted avg    0.98492   0.98598   0.98416     85811



We get a 51% recall and 65% f1-score for non-potable.  Not too bad (there is likely something there).

Now let us try a decision tree.

In [21]:
model = DecisionTreeClassifier(random_state=777)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
score_model(test_y, pred_y)

True Positive 83137 (Correctly classified potable)
True Negative: 1720 (Correctly classified non-potable)
False Positive 471 (Incorrectly classified non-potable as potable - Bad)
False Negative 483 (Incorrectly classified potable as non-potable - Meh)
              precision    recall  f1-score   support

         0.0    0.78075   0.78503   0.78289      2191
         1.0    0.99437   0.99422   0.99430     83620

    accuracy                        0.98888     85811
   macro avg    0.88756   0.88963   0.88859     85811
weighted avg    0.98891   0.98888   0.98890     85811



This impoves the recall/f1-score for non-potable to 78%.  Much better.

Now let us try a random forest.

In [22]:
model = RandomForestClassifier(random_state=777)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
score_model(test_y, pred_y)

True Positive 83488 (Correctly classified potable)
True Negative: 1686 (Correctly classified non-potable)
False Positive 505 (Incorrectly classified non-potable as potable - Bad)
False Negative 132 (Incorrectly classified potable as non-potable - Meh)
              precision    recall  f1-score   support

         0.0    0.92739   0.76951   0.84111      2191
         1.0    0.99399   0.99842   0.99620     83620

    accuracy                        0.99258     85811
   macro avg    0.96069   0.88397   0.91865     85811
weighted avg    0.99229   0.99258   0.99224     85811



And this raised the recall/f1-score for non-potable to 76%/84%.  Pretty good.

Now let us try to predict another parameter specific potability, this time for Dissolved Nitrate.

In [54]:
label_feature = "potable_Dissolved Nitrate"
df[label_feature] = df[label_feature].astype("category")
train_df, test_df = train_test_split(df, train_size=0.75, random_state=777, stratify=df[label_feature])
print(f"Train size: {train_df.shape}, Test size: {test_df.shape}")

scaler = StandardScaler()
num_features = [
    "latitude", "longitude",
    "Shape__Area", "Shape__Length",
    "emp", "qp1", "ap", "est",
    "n<5", "n5_9", "n10_19", "n20_49", "n50_99", "n100_249", "n250_499", "n500_999",
    "n1000", "n1000_1", "n1000_2", "n1000_3", "n1000_4",
    "Population"
] + df.columns[df.columns.str.contains("param_")].values.tolist()
num_features.remove("param_Dissolved Nitrate")
scaler.fit(train_df[num_features])
train_num = scaler.transform(train_df[num_features])
test_num = scaler.transform(test_df[num_features])

hot_enc = OneHotEncoder()
df.county_name = df.county_name.astype("category")
cat_features = ["county_name", "AdminRegion", "FireMAR", "LawMAR"]
hot_enc.fit(train_df[cat_features])
train_hot = hot_enc.transform(train_df[cat_features])
test_hot = hot_enc.transform(test_df[cat_features])

train_X = sp.hstack((train_hot, train_num))
train_y = train_df[label_feature].values
test_X = sp.hstack((test_hot, test_num))
test_y = test_df[label_feature].values
print(f"Train X {train_X.shape}, y {len(train_y)}")
print(f"Test X {test_X.shape}, y {len(test_y)}")

Train size: (257433, 899), Test size: (85811, 899)
Train X (257433, 529), y 257433
Test X (85811, 529), y 85811


Logistic

In [25]:
model = LogisticRegression(random_state=777, max_iter=10000)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
score_model(test_y, pred_y)

True Positive 83605 (Correctly classified potable)
True Negative: 166 (Correctly classified non-potable)
False Positive 1907 (Incorrectly classified non-potable as potable - Bad)
False Negative 133 (Incorrectly classified potable as non-potable - Meh)
              precision    recall  f1-score   support

         0.0    0.55518   0.08008   0.13997      2073
         1.0    0.97770   0.99841   0.98795     83738

    accuracy                        0.97623     85811
   macro avg    0.76644   0.53924   0.56396     85811
weighted avg    0.96749   0.97623   0.96746     85811



Awful.  We only had an 8% recall (13% f1-score).  Now for decision tree.

In [55]:
model = DecisionTreeClassifier(random_state=777)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
score_model(test_y, pred_y)

True Positive 82721 (Correctly classified potable)
True Negative: 1186 (Correctly classified non-potable)
False Positive 887 (Incorrectly classified non-potable as potable - Bad)
False Negative 1017 (Incorrectly classified potable as non-potable - Meh)
              precision    recall  f1-score   support

         0.0    0.53836   0.57212   0.55472      2073
         1.0    0.98939   0.98785   0.98862     83738

    accuracy                        0.97781     85811
   macro avg    0.76387   0.77999   0.77167     85811
weighted avg    0.97849   0.97781   0.97814     85811



Meh.  Better than a coin flip at 57%/55%.  And a random forest.

In [27]:
model = RandomForestClassifier(random_state=777)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
score_model(test_y, pred_y)

True Positive 83591 (Correctly classified potable)
True Negative: 948 (Correctly classified non-potable)
False Positive 1125 (Incorrectly classified non-potable as potable - Bad)
False Negative 147 (Incorrectly classified potable as non-potable - Meh)
              precision    recall  f1-score   support

         0.0    0.86575   0.45731   0.59848      2073
         1.0    0.98672   0.99824   0.99245     83738

    accuracy                        0.98518     85811
   macro avg    0.92624   0.72778   0.79547     85811
weighted avg    0.98380   0.98518   0.98293     85811



Worse - that dropped recall to 45% but raised f1-score to 59%.

Let us try to predict Dissolved Fluoride.

In [48]:
label_feature = "potable_Dissolved Fluoride"
df[label_feature] = df[label_feature].astype("category")
train_df, test_df = train_test_split(df, train_size=0.75, random_state=777, stratify=df[label_feature])
print(f"Train size: {train_df.shape}, Test size: {test_df.shape}")

scaler = StandardScaler()
num_features = [
    "latitude", "longitude",
    "Shape__Area", "Shape__Length",
    "emp", "qp1", "ap", "est",
    "n<5", "n5_9", "n10_19", "n20_49", "n50_99", "n100_249", "n250_499", "n500_999",
    "n1000", "n1000_1", "n1000_2", "n1000_3", "n1000_4",
    "Population"
] + df.columns[df.columns.str.contains("param_")].values.tolist()
num_features.remove("param_Dissolved Fluoride")
scaler.fit(train_df[num_features])
train_num = scaler.transform(train_df[num_features])
test_num = scaler.transform(test_df[num_features])

hot_enc = OneHotEncoder()
df.county_name = df.county_name.astype("category")
cat_features = ["county_name", "AdminRegion", "FireMAR", "LawMAR"]
hot_enc.fit(train_df[cat_features])
train_hot = hot_enc.transform(train_df[cat_features])
test_hot = hot_enc.transform(test_df[cat_features])

train_X = sp.hstack((train_hot, train_num))
train_y = train_df[label_feature].values
test_X = sp.hstack((test_hot, test_num))
test_y = test_df[label_feature].values
print(f"Train X {train_X.shape}, y {len(train_y)}")
print(f"Test X {test_X.shape}, y {len(test_y)}")

Train size: (257433, 899), Test size: (85811, 899)
Train X (257433, 529), y 257433
Test X (85811, 529), y 85811


Logistic

In [50]:
model = LogisticRegression(random_state=777, max_iter=10000)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
score_model(test_y, pred_y)

True Positive 84753 (Correctly classified potable)
True Negative: 46 (Correctly classified non-potable)
False Positive 963 (Incorrectly classified non-potable as potable - Bad)
False Negative 49 (Incorrectly classified potable as non-potable - Meh)
              precision    recall  f1-score   support

         0.0    0.48421   0.04559   0.08333      1009
         1.0    0.98877   0.99942   0.99407     84802

    accuracy                        0.98821     85811
   macro avg    0.73649   0.52251   0.53870     85811
weighted avg    0.98283   0.98821   0.98336     85811



Decision Tree

In [51]:
model = DecisionTreeClassifier(random_state=777)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
score_model(test_y, pred_y)

True Positive 84380 (Correctly classified potable)
True Negative: 596 (Correctly classified non-potable)
False Positive 413 (Incorrectly classified non-potable as potable - Bad)
False Negative 422 (Incorrectly classified potable as non-potable - Meh)
              precision    recall  f1-score   support

         0.0    0.58546   0.59068   0.58806      1009
         1.0    0.99513   0.99502   0.99508     84802

    accuracy                        0.99027     85811
   macro avg    0.79030   0.79285   0.79157     85811
weighted avg    0.99031   0.99027   0.99029     85811



Random Forest

In [52]:
model = RandomForestClassifier(random_state=777)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
score_model(test_y, pred_y)

True Positive 84732 (Correctly classified potable)
True Negative: 524 (Correctly classified non-potable)
False Positive 485 (Incorrectly classified non-potable as potable - Bad)
False Negative 70 (Incorrectly classified potable as non-potable - Meh)
              precision    recall  f1-score   support

         0.0    0.88215   0.51933   0.65377      1009
         1.0    0.99431   0.99917   0.99674     84802

    accuracy                        0.99353     85811
   macro avg    0.93823   0.75925   0.82525     85811
weighted avg    0.99299   0.99353   0.99270     85811



## Summary

While it should be theoretically possible to get a 95% accurate overall potability from trying to predict the potability of just the top predictors, the correlation to other parameters just doesn't appear to be there.  I am also not sure of a statistically valid way to combine these multiple models since each one is being trained on a different stratified boundary.